# 관광지 벡터 생성


1. 관광지 태그 데이터 전처리
- 비짓제주 태그 데이터, 네이버맵 테마키워드와 리뷰 정보에서 키워드 추출
- 리뷰 데이터 내 명사와 형용사만 추출
- 관광지 별 키워드를 합쳐 문서 생성
2. 관광지 별 TF-IDF 벡터 생성
- 관광지 문서에서 TF-IDF 벡터 생성 및 차원축소


In [23]:
import pandas as pd
from collections import Counter
import ast
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import ast
from konlpy.tag import Okt
from numpy import dot
from numpy.linalg import norm
import re

In [11]:
data = pd.read_csv('data/관광지_구역배정.csv', encoding='utf-8-sig')

data = data.dropna(subset=['tags']).reset_index(drop=True)
data_org = data.copy()

In [12]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

## 1. 관광지 태그 데이터 전처리


In [13]:
tags_list = [tag for tags in data['tags'] for tag in tags.split(' ')]

tags_dict = Counter(tags_list)
tags_dict = dict(sorted(tags_dict.items(), key=lambda item: item[1], reverse=True))

In [14]:
tags_dict

{'#어트랙션': 709,
 '#자연경관': 595,
 '#맑음': 405,
 '#체험': 350,
 '#어린이': 334,
 '#커플': 315,
 '#친구': 288,
 '#아이': 267,
 '#포토스팟': 247,
 '#레저/체험': 229,
 '#실내관광지': 221,
 '#문화관광': 216,
 '#역사유적': 201,
 '#도보': 200,
 '#도보여행': 199,
 '#여름': 161,
 '#경관/포토': 158,
 '#혼자': 151,
 '#해변': 151,
 '#휴식/힐링': 145,
 '#가을': 145,
 '#사계절': 137,
 '#액티비티': 137,
 '#흐림': 132,
 '#부모': 131,
 '#봄': 130,
 '#문화유적지': 118,
 '#상점/상가': 114,
 '#걷기/등산': 107,
 '#관광기념품': 107,
 '#겨울': 104,
 '#오름': 103,
 '#테마공원': 96,
 '#실내': 95,
 '#비.눈': 92,
 '#언택트': 90,
 '#쇼핑': 89,
 '#수상레저': 88,
 '#체험관광': 83,
 '#물놀이': 69,
 '#미술/박물관': 58,
 '#안전여행스탬프': 48,
 '#올레': 48,
 '#우수관광사업체': 46,
 '#박물관': 39,
 '#마을관광': 36,
 '#일몰': 36,
 '#봄꽃': 34,
 '#숲': 33,
 '#기념품': 32,
 '#휴식/치유': 31,
 '#청년': 31,
 '#포구': 30,
 '#마을산책': 29,
 '#일출': 28,
 '#드라이브': 26,
 '#밤': 24,
 '#한라산': 24,
 '': 24,
 '#중/장년': 24,
 '#유네스코': 23,
 '#캠핑': 23,
 '#해수욕장': 23,
 '#수국': 22,
 '#단풍': 22,
 '#유채꽃': 22,
 '#제주4.3': 20,
 '#동백': 19,
 '#예술': 19,
 '#숲길': 19,
 '#미술관': 19,
 '#승마': 18,
 '#곶자왈': 17,
 '#계곡': 17,

- 비슷한 분류라서 편입
    - #경관/포토 = ['#포토', '#포토스팟', '자연경관', '경관/포토']
    - #실내 = ['#실내관광지', '#실내', '#실내놀이터']
    - #4.3 = ['#제주4.3', '#4.3']
    - #비/바람/눈 = ['#비/바람/눈', '#비.눈', '#비']
    - #안개/흐림 = ['#흐림', '#안개/흐림']
    - #절 = ['#사찰', '#절']
    - #요가 = ['#요가', '#메디컬요가', '#플라잉요가', '#빈야사요가', '#하타요가', '#테랍툴요가', '#패들요가']
    - #낚시 = ['#선상낚시', '#낚시체험']
    - #체험 = ['#체험관광', '#체험프로그램', '#체험여행', '#자연염색', '#천연염색', '체험전시']
    - #레저/체험 = ['#레저/체험', '#레저', '#체험']
    - #수상레저 = [#waterleisure', '#수상레저', '#해상레저']
    - #휴식/힐링/치유 = ['#휴식', '#휴식/치유', '#휴식/힐링', '#치유', '#힐링']
    - #서핑 = ['#서핑', '#서핑스쿨']
    - #등산 = ['#등산', '#산']
    - #걷기/등산 - ['#마을산책', '#도보', '#도보여행', '#걷기', '등산', '산', '걷기/등산', '겯기/등산',  '걷기/휴식']
    - #포토스팟 = ['#우정스냅', '#커플스냅', '#포토스팟', '#사진']
    - #숲 = ['#숲', '#환상숲', '#숲길', '#휴양림']
    - #공연 = ['#공연장', '#대극장', '#연극', '#공연', '#뮤지컬']
    - #전시/행사 = ['전시와행사', '전시/행사', #전시관', '#전시', '체험전시']
    - #미술/박물관 = ['#미술', '#미술관', '#박물관', '#미술/박물관']
    - #유적지 = ['#유적지', '#문화유적지', '#역사유적지', '#역사', '#역사유적']
    - #어린이 = ['#아이', '#어린이', '#어린이동물농장']
    - #문화관광 = ['#문화관광', '#문화']
    - #사계절 = ['#사계절', '#사게절']
    - #해수욕장 = ['#해수욕장', '#곽지해수욕장']
- 의미 없어서 삭제
    - "''", '#구제주', '#제주시', '#고산리', '#의귀리', '#서귀포', '#수산리', '빵'

### 1-(2) 네이버 테마키워드 및 리뷰 데이터 정제

In [25]:
data = data[data['visitor_review_list']!='no_result'].reset_index(drop=True)

In [26]:
data['visitor_review_list'] = data['visitor_review_list'].fillna(int(0))
data['visitor_review'] = [ ast.literal_eval(dictionary) if type(dictionary)!=int else dictionary for dictionary in data['visitor_review_list']]
data['visitor_review'] = [list(dictionary.keys()) if type(dictionary)!=int else dictionary for dictionary in data['visitor_review']]

네이버 리뷰 데이터에서 명사와 형용사만 추출

In [27]:
review_all = [ review for review_list in data['visitor_review'] if type(review_list)!= int for review in review_list]
review_all = list(set(review_all))

In [24]:
okt = Okt()
review_dict = {key:[] for key in review_all}
for review in review_all:
    tmp = okt.pos(review)
    for pos in tmp:
        if (pos[1] == 'Noun') | (pos[1] == 'Adjective'):
            review_dict[review].append(pos[0])

NameError: name 'jpype' is not defined

In [28]:
for idx, row in enumerate(data['visitor_review']):
    if type(row) != int:
        tmp_list = []
        for sentence in row:
            tmp_list.extend(review_dict[sentence])
        data.at[idx, 'visitor_review'] = ' '.join(tmp_list)

NameError: name 'review_dict' is not defined

### 1-(3) 관광지의 리뷰 데이터와 테마키워드 합쳐서 문서로 생성

In [48]:
data['tag_all'] = [ str(mood) + ' ' + str(topic) + ' ' + str(why) + ' ' + str(visitor) for mood, topic, why, visitor in zip(data['dl_mood_list'], data['dl_topic_list'], data['dl_why_list'], data['visitor_review'])]
data['tag_all'] = data['tag_all'].apply(lambda x : x.replace(',', '').replace('0','').strip())

In [49]:
craw_tag_all = []
craw_tag_all = [ word for tag in data['tag_all']  for word in tag.split(' ') if word not in craw_tag_all]

### 1-(4) 태그 정제

In [53]:
data['tags'] = [ tags.replace('#','') for tags in data['tags'] ]
data['tag_all'] = data['tag_all'] + ' ' + data['tags']

In [54]:
# key가 value에 들어 있으면 안돼
change_dict = {'자연경관' : ['뷰', '경치', '경관', '경관/포토'],
               '포토스팟' : ['포토', '우정스냅', '커플스냅', '경관/포토'],
               '실내' : ['실내관광지', '실내놀이터'],
               '4.3' : ['제주4.3', ],
               '비/바람/눈' : ['비', '비.눈', '비올때', '비오는날'], '안개/흐림' : ['흐림'],
               '레저/체험' : ['레저', '체험', '카트체험'], '전시/행사' : ['체험전시', '전시', '행사', '전시와행사', '전시관'],
               '걷기/등산' : ['산책로', '산책', '산책길', '걷기', '등산', '겯기/등산', '걷기/휴식', '도보', '도보여행', '마을산책', '등산', '산'],
               '꽃' : ['수국', '유채꽃', '철쭉', '봄꽃'],
               '봄' : ['봄여행', '봄꽃'], '다양한' : ['다양해요'],
               '절' : ['사찰'] , '요가' : ['메디컬요가', '플라잉요가', '빈야사요가', '하타요가', '테랍툴요가', '패들요가'],
               '낚시' : ['선상낚시', '낚시체험'], '일출명소' : ['일출'], '일몰명소' : ['일몰'],
               '체험' : ['체험관광', '체험프로그램', '체험여행', '자연염색', '천연염색'], '수상레저' : ['waterleisure', '해상레저'],
               '휴식/힐링/치유' : ['휴식', '휴식/힐링', '휴식/치유', '치유', '힐링'],
               '서핑' : ['서핑스쿨'],
               '숲' : ['환상숲', '숲길', '휴양림', '편백나무숲'],
               '공연' : ['극장', '공연장', '대극장', '연극', '뮤지컬'], '미술관/박물관' : ['미술/박물관', '미술관', '박물관', '미술'],
               '유적지' : ['문화유적지', '역사유적지', '역사', '역사유적', '유적', '유물'],
               '어린이' : ['아이', '어린이동물농장'], '재미있는' : ['재밌어요'],
               '문화관광' : ['문화'], '사계절' : ['사게절'], '해수욕장' : ['곽지해수욕장'],
               '빵' : ['감귤상웨빵', '거시기빵'],
               '신선한' : ['신선해요']}

In [55]:
del_list = ['구제주', '제주시', '고산리', '의귀리', '서귀포', '수산리', '안개/흐림', '비/바람/눈', '맑음', '커플', '친구', '어린이', '부모', '청년', '중/장년',
            '혼자', '휴식/힐링/치유', '걷기/등산', '자연경관', '어트랙션', '레저/체험', '문화관광', '해변']
data['tags_list'] = data['tag_all'].apply(lambda x: tag_cleaning(x, change_dict, del_list))

##  2. 관광지 TF-IDF 벡터 생성


In [56]:
## TF-IDF로 관광지 벡터를 만들어보자!
#### TF-IDF에 태그 빼주기
del_list = ['안개/흐림', '비/바람/눈', '맑음', '커플', '친구', '어린이', '부모', '청년', '중/장년', '혼자', '휴식/힐링/치유', '걷기/등산', '자연경관', '어트랙션',
            '레저/체험', '문화관광', '해변']
for idx, row in enumerate(data['tags_list']):
    row_list = []
    for word in row:
        if word not in del_list:
            row_list.append(word)
    data.at[idx, 'tag_all_vj_no_no'] = ' '.join(row_list)

'편해요 박람회 가격 겨울 한라봉 구성 주차 합리 승마체험 미술관/박물관 재미있는 화려한 다양한 이국적 실내 폭포 유익해요 식물원 알차요 유적지 가족여행 전시/행사 프로그램 조그마한'

In [ ]:
# 필요 없는 태그 정제

In [ ]:
cluster_tags = data.groupby('cluster')['tags_list'].apply(lambda x: x.sum()).to_frame().reset_index()
final_list = []
for row in cluster_tags['tags_list']:
    row_list = []
    for word in row:
        if word not in del_list:
            row_list.append(word)
    final_list.append(row_list)
cluster_tags['tag_all_vj_no_no'] = final_list
cluster_tags['tag_all_vj_no_no'] = cluster_tags['tag_all_vj_no_no'].apply(' '.join)

In [ ]:
# TF IDF  벡터 만들기

In [ ]:
tfidf_plc = TfidfVectorizer()
place_plc = tfidf_plc.fit_transform(data['tag_all_vj_no_no'])
cluster_plc = tfidf_plc.transform(cluster_tags['tag_all_vj_no_no'])

In [ ]:
# place 기준으로 PCA
svd_place = TruncatedSVD(n_components=220)
svd_place_plc = svd_place.fit_transform(place_plc) # fit tranform
svd_cluster_plc = svd_place.transform(cluster_plc) # transform
var_explained_plc = svd_place.explained_variance_ratio_.sum()
var_explained_plc  # dimension 220이면 .93

In [ ]:
# 행렬 만듦 (place vector - plc로 pca / cluter vector - plc로 pca)
svd_place_plc_df = pd.DataFrame(svd_place_plc, columns = [i for i in range(svd_place.n_components)])
svd_cluster_plc_df = pd.DataFrame(svd_cluster_plc, columns = [i for i in range(svd_place.n_components)])

In [ ]:
svd_place_plc_df['similarity'] = 0
svd_place_plc_df['similarity'] = [ cos_sim(svd_cluster_plc_df.iloc[cls], svd_place_plc_df.iloc[idx][0:220]) if cls<2 else cos_sim(svd_cluster_plc_df.iloc[cls-1], svd_place_plc_df.iloc[idx][0:220])  for idx, cls in enumerate(svd_place_plc_df['cluster']) ] # 유사도 계산

In [57]:
svd_place_plc_df['blog_review_num_list'] = data['blog_review_num_list'] # 필요한 column정리
svd_place_plc_df[['co2_max', 'lon', 'lat']] = data[['co2_max', 'GPS_LNGTD', 'GPS_LTTD']]
svd_place_plc_df = svd_place_plc_df[['cluster', 'name_PRE', 'similarity', 'co2_max', 'lon', 'lat', 'blog_review_num_list']]

NameError: name 'svd_place_plc_df' is not defined

In [ ]:
svd_place_plc_df.to_csv('data/관광지_유사도.csv', encoding='utf-8-sig', index=False)

## 3. 관광지 추천점수 산정

관광지 추천점수 5점 척도
- eco_rate : 바람지도/푸른컵 등 친환경 사업 여부 1 or 0
- co2_rate : 건물에 대한 탄소 배출량 스케일
- review_rate : 네이버 블로그 수 스케일
- cluster_rate : 구역의 특성과 일치하는 정도
- dist_rate : 선별된 F&B와의 거리


In [30]:
category = data
data = svd_place_plc_df

NameError: name 'svd_place_plc_df' is not defined

In [ ]:
category = category[['name_PRE','CL_NM','CAT_PRE','PREICE_MAX']]
category.columns = ['name_PRE','CL_NM','CAT_PRE','price']
category.price[category.price.isna()] = 0

data = pd.merge(left=data, right=category, how="left", on = "name_PRE")
data = data[['cluster', 'CL_NM', 'CAT_PRE','name_PRE', 'lon', 'lat','price','blog_review_num_list','co2_max', 'similarity']]
data['type'] = "관광지"
data

#### 1. 친환경 사업 여부


In [31]:
baram = pd.read_csv('data/제주바람지도.csv', encoding='utf-8-sig')
bluecup = pd.read_csv('data/제주푸른컵지도.csv', encoding='utf-8-sig')

In [32]:
# 이름 전처리용
def name_preprocessing(name):
    name = name.replace(' (', '(').replace(') ', ')').strip()
    name = name.replace('(주)','').replace("·",".").lower()
    name = re.sub('[\(\)㈜!@#$%^&*{}|:;\'\"/><,~`+=_ ]','', name)
    name = re.sub('[\[.\]]','',name)
    return name.strip()

#주소 전처리용
def addr_preprocessing(addr):
    addr = re.sub('제주 서귀포시','제주특별자치도 서귀포시',addr)
    addr = re.sub('제주 제주시','제주특별자치도 제주시',addr)

    while (addr.split()[-1][-1].isdigit() == False) : #상세주소 제거
        if len(addr.split()) <= 3 : break
            
        addr = " ".join(addr.split()[0:-1]) #하나빼기

    addr = "".join(addr.split()) #공백다 없애기
    return addr.strip() #앞뒤정리

In [33]:
baram.columns = ['area_name', 'tel', 'instagram', 'detail', 'category']

In [34]:
eco_list = []
        
#이름이 일치하는 경우
name_list = list(map(lambda x : name_preprocessing(x), baram['area_name']))
set(eco_list).union(set(data[data['name_PRE'].isin(name_list)].index))

#포함되는 경우
for i in range(0, len(name_list)) :
    idx = data[data['name_PRE'].str.contains(name_list[i])].index
    if len(idx) >0 :
        eco_list.append(idx.values[0])

for i in range(0, len(data)) :
    if len(baram[baram['area_name'].str.contains(data.name_PRE[i])]) >0 : 
        eco_list.append(i)

NameError: name 're' is not defined

In [ ]:
#이름이 일치하는 경우
name_list = list(map(lambda x : name_preprocessing(x), bluecup['이름']))
set(eco_list).union(set(data[data['name_PRE'].isin(name_list)].index))

# 초함되는 경우
for i in range(0, len(name_list)) :
    idx = data[data['name_PRE'].str.contains(name_list[i])].index
    if len(idx) >0 :
        eco_list.append(idx.values[0])

for i in range(0, len(data)) :
    if len(bluecup[bluecup['이름'].str.contains(data.name_PRE[i])]) >0 : 
        eco_list.append(i)

In [ ]:
#중복제거 및 처리
eco_list = set(eco_list)
data.iloc[list(eco_list)]

In [ ]:
data['eco_YN'] = 0
data['eco_YN'].iloc[list(eco_list)] = 1

#### 2. 건물에 대한 탄소배출량 스케일 점수

In [ ]:
data['co2_rate'] = (1 - minmax_scale(data['co2_max'],copy=True)).tolist()

In [35]:
data['co2_rate'].sort_values()

KeyError: 'co2_rate'

In [ ]:
import seaborn as sns
sns.distplot(data['co2_rate'])

### (3) 네이버 블로그 수

In [ ]:
data['review_rate'] = minmax_scale(data['blog_review_num_list'],copy=True).tolist()

In [ ]:
data['review_rate'].sort_values()

In [ ]:
sns.distplot(data['review_rate'])

### (4) 선정된 F&B와의 접근성

In [ ]:
#클러스터별로 나누기
data0 = data[data.cluster==0].reset_index(drop=True)
data1 = data[data.cluster==1].reset_index(drop=True)
data3 = data[data.cluster==3].reset_index(drop=True)
data4 = data[data.cluster==4].reset_index(drop=True)
data5 = data[data.cluster==5].reset_index(drop=True)

In [ ]:
# F&B선별 데이터
fnb0 = pd.read_csv('cluster0최종FB.csv', encoding='utf-8-sig')
fnb1 = pd.read_csv('cluster1최종FB.csv', encoding='utf-8-sig')
fnb3 = pd.read_csv('cluster3최종FB.csv', encoding='utf-8-sig')
fnb4 = pd.read_csv('cluster4최종FB.csv', encoding='utf-8-sig')
fnb5 = pd.read_csv('cluster5최종FB.csv', encoding='utf-8-sig')

In [ ]:
#계산할 관광지 후보
def calcuate_dist(data,fnb):
    calcul_df = data[['name_PRE','lat','lon']]
    
    for i in range(0,len(fnb)):
        target = tuple(fnb[['lat','lon']].iloc[i])
        dist_list = []
    
        for j in range(0,len(calcul_df)):
            distance = haversine(target, tuple(calcul_df.iloc[j,1:3]), unit = 'km')
            dist_list.append(distance)
        calcul_df[f'{i}'] = dist_list
    #거리 최소값 계산
    calcul_df['dist_min'] = calcul_df.iloc[:,3:].min(axis=1)

    # 4km 이하인 거리는 모두 같은것(4km)으로 간주
    calcul_df['dist_min'][calcul_df['dist_min'] <= 4] = 4

    # Normalization > 점수부여
    data['dist_score'] = (1 - minmax_scale(calcul_df['dist_min'], copy=True)).tolist()

    return(data)

In [ ]:
data0 = calcuate_dist(data0,fnb0)
data1 = calcuate_dist(data1,fnb1)
data3 = calcuate_dist(data3,fnb3)
data4 = calcuate_dist(data4,fnb4)
data5 = calcuate_dist(data5,fnb5)

#### (5) 구역 특징과의 유사도

In [ ]:
data0['total_score'] = data0[[ 'eco_YN', 'review_rate', 'co2_rate','dist_score', 'similarity']].sum(axis=1)
data0.sort_values(['total_score'],ascending = False )[:8]

In [ ]:
data1['total_score'] = data1[[ 'eco_YN', 'review_rate', 'co2_rate','dist_score', 'similarity']].sum(axis=1)
data1.sort_values(['total_score'],ascending = False )[:10]

In [ ]:
data3['total_score'] = data3[[ 'eco_YN', 'review_rate', 'co2_rate','dist_score', 'similarity']].sum(axis=1)
data3.sort_values(['total_score'],ascending = False )[:8]

In [ ]:
data4['total_score'] = data4[[ 'eco_YN', 'review_rate', 'co2_rate','dist_score', 'similarity']].sum(axis=1)
data4.sort_values(['total_score'],ascending = False )[:10]

In [ ]:
data5['total_score'] = data5[[ 'eco_YN', 'review_rate', 'co2_rate','dist_score', 'similarity']].sum(axis=1)
data5.sort_values(['total_score'],ascending = False )[:10]

In [ ]:
all_visits = pd.concat([
    data0.sort_values(['total_score'],ascending = False )[:8],
    data1.sort_values(['total_score'],ascending = False )[:10],
    data3.sort_values(['total_score'],ascending = False )[:8],
    data4.sort_values(['total_score'],ascending = False )[:10],
    data5.sort_values(['total_score'],ascending = False )[:8]
])
all_visits

In [ ]:
all_visits.shape

In [ ]:
fnb0['cluster'] = 0
fnb1['cluster'] = 1
fnb3['cluster'] = 3
fnb4['cluster'] = 4
fnb5['cluster'] = 5

fnb0['type'] = "F&B"
fnb1['type'] = "F&B"
fnb3['type'] = "F&B"
fnb4['type'] = "F&B"
fnb5['type'] = "F&B"

In [ ]:
final.to_csv('관광지F&B선정완료_FINAL.csv', encoding='utf-8-sig', index=False) 

In [ ]:
## 지도 시각화
center = [33.3618, 126.5291] #한라산
m = folium.Map(location=center, zoom_start=10, tiles='cartodbpositron')
style = {'fillColor': '#F15F5F', 'color': '#F15F5F'}

df = final

for i in range(len(df)):
    folium.Circle(
        location=[df.iloc[i]['lat'], df.iloc[i]['lon']],
        popup=df.iloc[i]['AREA_NM_PRE'],
        radius=100,
        fill=True, fill_color=color_with(df.iloc[i]['cluster']), opacity=0.5,
        color=color_with(df.iloc[i]['cluster']),
    ).add_to(m)
    
m

In [ ]:
fig = px.scatter(final, 
                 x="lon", 
                 y="lat",
                 color = 'cluster',
                 hover_name = 'cluster',
                 hover_data=["AREA_NM_PRE"])
fig.show()